In [2]:
df2 = pd.read_csv("/content/UNSW_preview.csv")

In [3]:
list(df2.columns)


['dur',
 'proto',
 'service',
 'state',
 'spkts',
 'dpkts',
 'sbytes',
 'dbytes',
 'rate',
 'sload',
 'dload',
 'sloss',
 'dloss',
 'sinpkt',
 'dinpkt',
 'sjit',
 'djit',
 'swin',
 'stcpb',
 'dtcpb',
 'dwin',
 'tcprtt',
 'synack',
 'ackdat',
 'smean',
 'dmean',
 'trans_depth',
 'response_body_len',
 'ct_src_dport_ltm',
 'ct_dst_sport_ltm',
 'is_ftp_login',
 'ct_ftp_cmd',
 'ct_flw_http_mthd',
 'is_sm_ips_ports',
 'attack_cat',
 'label']

In [39]:
df = pd.read_csv("/content/Testing.csv")
df['proto'].unique()


array(['udp', 'arp', 'tcp', 'igmp', 'ospf', 'sctp', 'gre', 'ggp', 'ip',
       'ipnip', 'st2', 'argus', 'chaos', 'egp', 'emcon', 'nvp', 'pup',
       'xnet', 'mux', 'dcn', 'hmp', 'prm', 'trunk-1', 'trunk-2',
       'xns-idp', 'leaf-1', 'leaf-2', 'irtp', 'rdp', 'netblt', 'mfe-nsp',
       'merit-inp', '3pc', 'idpr', 'ddp', 'idpr-cmtp', 'tp++', 'ipv6',
       'sdrp', 'ipv6-frag', 'ipv6-route', 'idrp', 'mhrp', 'i-nlsp', 'rvd',
       'mobile', 'narp', 'skip', 'tlsp', 'ipv6-no', 'any', 'ipv6-opts',
       'cftp', 'sat-expak', 'ippc', 'kryptolan', 'sat-mon', 'cpnx', 'wsn',
       'pvp', 'br-sat-mon', 'sun-nd', 'wb-mon', 'vmtp', 'ttp', 'vines',
       'nsfnet-igp', 'dgp', 'eigrp', 'tcf', 'sprite-rpc', 'larp', 'mtp',
       'ax.25', 'ipip', 'aes-sp3-d', 'micp', 'encap', 'pri-enc', 'gmtp',
       'ifmp', 'pnni', 'qnx', 'scps', 'cbt', 'bbn-rcc', 'igp', 'bna',
       'swipe', 'visa', 'ipcv', 'cphb', 'iso-tp4', 'wb-expak', 'sep',
       'secure-vmtp', 'xtp', 'il', 'rsvp', 'unas', 'fc', 'iso-ip',


In [5]:
import pandas as pd

df = pd.read_parquet("/content/UNSW_NB15_testing-set.parquet")
df.to_csv("Testing.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import joblib
import warnings
warnings.filterwarnings("ignore")


train_df = pd.read_csv('/content/Training.csv')
test_df  = pd.read_csv('/content/Testing.csv')


train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df  = test_df.sample(frac=1, random_state=42).reset_index(drop=True)



X_train = train_df.drop(['attack_cat', 'label'], axis=1)
y_train = train_df['attack_cat']

X_test = test_df.drop(['attack_cat', 'label'], axis=1)
y_test = test_df['attack_cat']


label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)


categorical_cols = ['proto', 'service', 'state']
numeric_cols = [col for col in X_train.columns if col not in categorical_cols]


preprocessor = ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ('scale', StandardScaler(), numeric_cols)
])


models = {
    "logistic_regression": LogisticRegression(max_iter=500),
    "decision_tree": DecisionTreeClassifier(),
    "random_forest": RandomForestClassifier(n_estimators=120),
    "xgboost": XGBClassifier(eval_metric='mlogloss', n_estimators=120)
}


saved_files = []

for model_name, model in models.items():

    print(f"\n Training Model: {model_name.upper()}")

    pipeline = Pipeline([
        ('preprocess', preprocessor),
        ('model', model)
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


    filename = f"{model_name}_cyber_classifier.pkl"
    joblib.dump({"model": pipeline, "label_encoder": label_encoder}, filename)

    saved_files.append(filename)
    print(f"💾 Saved: {filename}")


print(" ALL MODELS TRAINED & SAVED SUCCESSFULLY")
for file in saved_files:
    print("Saved", file)



🔹 Training Model: LOGISTIC_REGRESSION
✔ Accuracy: 0.6709
                precision    recall  f1-score   support

      Analysis       0.00      0.00      0.00       677
      Backdoor       0.00      0.00      0.00       583
           DoS       0.25      0.02      0.03      4089
      Exploits       0.48      0.74      0.58     11132
       Fuzzers       0.22      0.52      0.31      6062
       Generic       0.99      0.96      0.98     18871
        Normal       0.89      0.62      0.73     37000
Reconnaissance       0.45      0.73      0.56      3496
     Shellcode       0.00      0.00      0.00       378
         Worms       0.00      0.00      0.00        44

      accuracy                           0.67     82332
     macro avg       0.33      0.36      0.32     82332
  weighted avg       0.74      0.67      0.68     82332

💾 Saved: logistic_regression_cyber_classifier.pkl

🔹 Training Model: DECISION_TREE
✔ Accuracy: 0.7474
                precision    recall  f1-score   suppo

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import joblib
import os

os.makedirs("model_metrics", exist_ok=True)
os.makedirs("confusion_matrices", exist_ok=True)

test_df = pd.read_csv("/content/Testing.csv")

y_true = test_df['attack_cat']
label_map = joblib.load("logistic_regression_cyber_classifier.pkl")['label_encoder']

models = {
    "logistic_regression": "/content/logistic_regression_cyber_classifier.pkl",
    "decision_tree": "/content/decision_tree_cyber_classifier.pkl",
    "random_forest": "/content/random_forest_cyber_classifier.pkl",
    "xgboost": "/content/xgboost_cyber_classifier.pkl"
}

results = []

for name, file in models.items():
    model_data = joblib.load(file)
    model = model_data['model']

    X_test = test_df.drop(['attack_cat', 'label'], axis=1)
    y_pred = label_map.inverse_transform(model.predict(X_test))

    cm = confusion_matrix(y_true, y_pred, labels=label_map.classes_)

    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="viridis", xticklabels=label_map.classes_, yticklabels=label_map.classes_)
    plt.title(f"{name} Confusion Matrix")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.savefig(f"confusion_matrices/{name}_confusion_matrix.png")
    plt.close()

    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, average="macro", zero_division=0),
        "Recall": recall_score(y_true, y_pred, average="macro", zero_division=0),
        "F1 Score": f1_score(y_true, y_pred, average="macro", zero_division=0)
    })

results_df = pd.DataFrame(results)
results_df.to_excel("model_metrics/model_scores.xlsx", index=False)

print("Metrics saved and confusion matrices generated.")


Metrics saved and confusion matrices generated.


In [11]:
!pip install crewai groq pandas joblib scikit-learn shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 641.8/641.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.

In [ ]:
import os, joblib, shap, pandas as pd, numpy as np
from crewai import Agent, Crew
from groq import Groq

os.environ["GROQ_API_KEY"] = ""
llm = Groq(api_key=os.environ["GROQ_API_KEY"])

MODEL_PATH = "/content/random_forest_cyber_classifier.pkl"
loaded = joblib.load(MODEL_PATH)
model = loaded['model']
label_encoder = loaded['label_encoder']
print(f"Loaded model: {MODEL_PATH}")

Watcher_Agent = Agent(
    role='Watcher',
    goal='Continuously monitor incoming network flow data and feed it into the Analyst Agent for prediction.',
    backstory=(
        "You are the Watcher Agent — a vigilant observer of all network activity. "
        "You detect new logs or packets arriving from sensors or datasets. "
        "Your task is to parse and forward these observations to the Analyst Agent."
    ),
    verbose=True,
    allow_delegation=True,
    llm=llm
)

Analyst_Agent = Agent(
    role='Analyst',
    goal='Use the pre-trained classical ML model to classify incoming samples as normal or anomalous.',
    backstory=(
        "You are the Analyst Agent — the intelligence behind the cyber defense system. "
        "You use a trained ML model to analyze each incoming event and predict whether it represents an attack or benign activity. "
        "You then pass this result to the Remediator Agent."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

Remediator_Agent = Agent(
    role='Remediator',
    goal='Take corrective actions when an anomaly is detected, such as isolating nodes or alerting admins.',
    backstory=(
        "You are the Remediator Agent — the protector of the network. "
        "When the Analyst flags a potential threat, you take swift action. "
        "Depending on severity, you can isolate infected nodes, update firewall rules, or trigger alerts. "
        "All actions are logged for transparency."
    ),
    verbose=True,
    allow_delegation=True,
    llm=llm
)

Explainer_Agent = Agent(
    role='Explainer',
    goal='Generate explainable insights for each detection using SHAP, helping analysts understand model reasoning.',
    backstory=(
        "You are the Explainer Agent — a transparent communicator between AI models and human analysts. "
        "Your job is to clarify why a sample was marked as a threat. "
        "You highlight key features like SYNFlag Count or Flow Duration that influenced the model’s decision."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

crew = Crew(agents=[Watcher_Agent, Analyst_Agent, Remediator_Agent, Explainer_Agent])

def analyze_event(new_data: pd.DataFrame):
    print("Watcher: received new event.")
    features = new_data.copy()

    pred = model.predict(features)
    decoded = label_encoder.inverse_transform(pred)[0]
    print("Analyst: prediction =", decoded)

    if decoded != "Normal":
        remediation = f"Alert: {decoded} attack detected. Executing mitigation and isolation protocol."
    else:
        remediation = "No threat detected. Continue monitoring."
    print("Remediator:", remediation)

    preprocess = model.named_steps['preprocess']
    inner_model = model.named_steps['model']

    transformed = preprocess.transform(features)

    if hasattr(transformed, "toarray"):
        transformed = transformed.toarray()

    transformed = transformed.astype(float)

    explainer = shap.TreeExplainer(inner_model, feature_perturbation="interventional")
    shap_values = explainer.shap_values(transformed, check_additivity=False)

    if isinstance(shap_values, list):
        shap_values = shap_values[np.argmax(inner_model.predict_proba(transformed)[0])]

    ohe_cols = model.named_steps['preprocess'].named_transformers_['onehot'].get_feature_names_out(['proto','service','state'])
    remaining_cols = [c for c in features.columns if c not in ['proto','service','state']]
    final_feature_names = list(ohe_cols) + remaining_cols

    importance_scores = np.abs(shap_values).mean(axis=0)
    top_indices = np.argsort(importance_scores)[::-1][:5]
    top_indices = np.array(top_indices).flatten().tolist()

    top_features = [final_feature_names[i] for i in top_indices]

    explanation = f"Top influencing features: {top_features}"
    print("Explainer:", explanation)

    return {
        "prediction": decoded,
        "remediation": remediation,
        "explanation": explanation
    }





Loaded model: /content/random_forest_cyber_classifier.pkl


In [36]:
df = pd.read_csv("/content/Testing.csv")
sample = df.drop(['attack_cat','label'], axis=1).iloc[[0]]
output = analyze_event(sample)
print(output)


Watcher: received new event.
Analyst: prediction = Shellcode
Remediator: Alert: Shellcode attack detected. Executing mitigation and isolation protocol.
Explainer: Top influencing features: ['proto_bna', 'proto_bbn-rcc', 'proto_arp', 'proto_aris', 'proto_argus', 'proto_ax.25', 'proto_3pc', 'proto_a/n', 'proto_any', 'proto_aes-sp3-d', 'proto_bna', 'proto_3pc', 'proto_aris', 'proto_ax.25', 'proto_arp', 'proto_bbn-rcc', 'proto_a/n', 'proto_argus', 'proto_aes-sp3-d', 'proto_any', 'proto_bna', 'proto_3pc', 'proto_a/n', 'proto_ax.25', 'proto_arp', 'proto_argus', 'proto_bbn-rcc', 'proto_aes-sp3-d', 'proto_aris', 'proto_any', 'proto_bna', 'proto_bbn-rcc', 'proto_arp', 'proto_a/n', 'proto_aris', 'proto_aes-sp3-d', 'proto_argus', 'proto_ax.25', 'proto_3pc', 'proto_any', 'proto_bna', 'proto_arp', 'proto_bbn-rcc', 'proto_a/n', 'proto_argus', 'proto_aris', 'proto_3pc', 'proto_any', 'proto_ax.25', 'proto_aes-sp3-d']
None
